## Q-Learning  
### 计算步骤：

1. Q-Table的创建： 想象一个表格，行是小狮子的位置，列是它可以采取的动作（比如上、下、左、右）。每个单元格的值叫Q值，代表在这个位置采取这个动作的“好坏”。
2. 更新Q值：

* 小狮子选择一个动作，移动到下一个位置。
* 得到奖励（如果找到火腿，加分；碰到陷阱，减分）。
* 更新公式是：
  新的Q值=旧的Q值+学习率×(奖励+未来最大Q值−旧的Q值)
* 学习率决定了这次更新对旧值的影响有多大。
## Linear Q-Learning
### 计算步骤：

1. 特征向量： 这里不再用表格，而是用一个简单的数学公式来表示Q值。想象小狮子的状态（位置）用几个数字来描述。
2. 更新权重：

* 使用特征向量来计算Q值。
* 更新权重的方式类似：
  新的权重=旧的权重+学习率×(奖励+未来最大Q值−当前Q值)×特征向量

* 这里的“特征向量”就像是描述小狮子当前状态的数字组合。
## 区别总结
1. Q-Learning： 使用表格逐个记录每个状态和动作的Q值，每次更新一个单元格。
2. Linear Q-Learning： 不用表格，而是用公式和数字组合来快速估算Q值，每次调整公式的参数。

## 详解特征向量和权重向量：

ϕ(s)=[x,y,d]<br>

x,y坐标，d是距离火腿的距离，实际上也可以d,d，两个距离都放进来。<br>

### 权重向量和特征向量:<br>

1. 特征向量： 代表当前状态的特征，通常是一个数组，如:<br>

ϕ(s)=[a,b,c,d...]。<br>

2. 权重向量： 存储与特征相关的权重，形状与特征向量相同，如 
𝜃=[a1,b1,c1,d1...]。<br>

权重向量就是Q-Table的Q-value,特征向量就是Q-Table,不过，特征向量和权重向量做了更精细的维度分解，x,y都有q-value,还有d,而不是one-hot了。<br>

特征更复杂，维度更高的情况下，Q-Table会报废，而Linear-Qlearning会更能站一些。<br>

In [1]:
import numpy as np
import random
import time
import matplotlib.pyplot as plt
from IPython import display

# 设置地图和相关参数
grid_size = 5
num_episodes = 10
max_steps = 20
learning_rate = 0.1
discount_factor = 0.9
exploration_rate = 1.0
exploration_decay = 0.99

# 初始化Q-Table
q_table = np.zeros((grid_size, grid_size, 4))  # 4个动作：上、下、左、右

# 定义动作
UP, DOWN, LEFT, RIGHT = 0, 1, 2, 3

# 定义环境状态
fire_position = (3, 2)
trap_position = (2, 1)


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
def get_next_position(state, action):
    x, y = state
    if action == UP:
        return (max(0, x - 1), y)
    elif action == DOWN:
        return (min(grid_size - 1, x + 1), y)
    elif action == LEFT:
        return (x, max(0, y - 1))
    elif action == RIGHT:
        return (x, min(grid_size - 1, y + 1))

def get_reward(state):
    if state == fire_position:
        return 10  # 找到火腿
    elif state == trap_position:
        return -10  # 碰到陷阱
    else:
        return -1  # 每一步的惩罚

def show_state(end, state, episode, step, q_table):
    terminal = fire_position
    hole = trap_position
    env = np.array([["_ "] * grid_size for _ in range(grid_size)])
    env[terminal] = "$ "
    env[hole] = "# "
    env[state] = "L "
    interaction = ""
    for row in env:
        interaction += "".join(row) + "\n"

    if state == terminal:
        message = "EPISODE: {}, STEP: {}".format(episode, step)
        interaction += message
        display.clear_output(wait=True)
        print(interaction)
        print("\n" + "q_table:")
        print(q_table)
        time.sleep(3)
    else:
        display.clear_output(wait=True)
        print(interaction)
        print(q_table)
        time.sleep(0.3)

# 训练过程
for episode in range(num_episodes):
    state = (0, 0)  # 初始化位置
    for step in range(max_steps):
        # 选择动作
        if random.uniform(0, 1) < exploration_rate:
            action = random.randint(0, 3)  # 随机选择动作
        else:
            action = np.argmax(q_table[state[0], state[1]])  # 选择Q值最高的动作

        next_state = get_next_position(state, action)
        reward = get_reward(next_state)

        # Q-Learning更新
        q_table[state[0], state[1], action] += learning_rate * (
            reward + discount_factor * np.max(q_table[next_state[0], next_state[1]]) - q_table[state[0], state[1], action]
        )

        # 更新状态
        state = next_state

        # 显示状态
        show_state(state == fire_position, state, episode, step, q_table)

    # 降低探索率
    exploration_rate *= exploration_decay

# 训练完成
print("Training finished!")
